In [75]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


train_data = pd.read_csv('train_features_withPBDeepFRI.csv',index_col=0)
test_data = pd.read_csv('test_features_withPBDeepFRI.csv',index_col=0)

X_train = train_data.drop('Label',axis = 1).to_numpy()
X_test = test_data.drop('Label',axis = 1).to_numpy()
y_train = train_data['Label']
y_test = test_data['Label']

In [76]:
num_timesteps = 1
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, num_features)
X_test_reshaped = X_test.reshape(-1, num_features)

X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(2518, num_timesteps, num_features)
X_test_scaled = scaler.transform(X_test_reshaped).reshape(630, num_timesteps, num_features)

In [84]:
model = Sequential()
model.add(SimpleRNN(units=128, input_shape=(num_timesteps, num_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

In [94]:
early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor the validation loss
    patience=10,            # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [100]:
# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [101]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32,
                    validation_split=0.1, callbacks=[early_stopping])

Epoch 1/50
71/71 [==============================] - 2s 10ms/step - loss: 0.4218 - accuracy: 0.8248 - val_loss: 0.3693 - val_accuracy: 0.8452
Epoch 2/50
71/71 [==============================] - 0s 6ms/step - loss: 0.4029 - accuracy: 0.8279 - val_loss: 0.3783 - val_accuracy: 0.8492
Epoch 3/50
71/71 [==============================] - 1s 7ms/step - loss: 0.3900 - accuracy: 0.8354 - val_loss: 0.4451 - val_accuracy: 0.8175
Epoch 4/50
71/71 [==============================] - 1s 7ms/step - loss: 0.3958 - accuracy: 0.8319 - val_loss: 0.4345 - val_accuracy: 0.8413
Epoch 5/50
71/71 [==============================] - 1s 8ms/step - loss: 0.3777 - accuracy: 0.8425 - val_loss: 0.4501 - val_accuracy: 0.8175
Epoch 6/50
71/71 [==============================] - 1s 8ms/step - loss: 0.3696 - accuracy: 0.8473 - val_loss: 0.4558 - val_accuracy: 0.7976
Epoch 7/50
71/71 [==============================] - 1s 8ms/step - loss: 0.4134 - accuracy: 0.8164 - val_loss: 0.4401 - val_accuracy: 0.8016
Epoch 8/50
71/71 [=

In [102]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

print(f"Test Accuracy: {test_accuracy:.4f}")


20/20 [==============================] - 0s 3ms/step - loss: 0.4269 - accuracy: 0.8175
Test Accuracy: 0.8175


In [103]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

# Assuming your model's predictions are probabilities and you want to binarize them with a threshold of 0.5
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)  # Note: use y_pred_prob for AUC as it requires probabilities

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'Matthews Correlation Coefficient: {mcc:.4f}')
print(f'AUC: {auc:.4f}')


20/20 [==============================] - 0s 2ms/step
Accuracy: 0.8175
Precision: 0.7660
Recall: 0.9143
F1-Score: 0.8336
Matthews Correlation Coefficient: 0.6472
AUC: 0.9093
